In [ ]:
from astropy.table import Table
from astropy.nddata import CCDData, Cutout2D
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from skimage import measure

In [ ]:
import warnings
from scipy.optimize import minimize

In [ ]:
imgdir = "../yasone2/img_i_01"
img = CCDData.read(imgdir + "/nobkg.fits", unit="adu")
mask = CCDData.read(imgdir + "/flag.fits", unit="adu")
img_err = CCDData.read(imgdir + "/flat_fielded.weight.fits", unit="adu")

img_masked = img.data.copy()
img_masked[mask.data == 1] = np.nan
img_masked[mask.data >= 2 ]  = np.nan

In [ ]:
SAT_LEVEL=58_000

In [ ]:
sat_pixels = mask.data == 1

In [ ]:

plt.imshow(sat_pixels)

In [ ]:
regions = measure.label(sat_pixels)

In [ ]:
props = measure.regionprops(regions)[100]

In [ ]:
props.centroid

In [ ]:
cutouts = []
Ny, Nx = img.shape
cutout_size = 50
for region in measure.regionprops(regions):
    cen = region.centroid

    if region.area < 100:
        continue
    xcen, ycen = np.int64(np.round(cen))
    r = min([cutout_size, xcen, ycen, Nx-xcen, Ny-ycen])
    xmin = max(xcen - r, 0)
    xmax = min(xcen + r, Nx-1)
    ymin = max(ycen - r, 0)
    ymax = min(ycen + r, Ny-1)
    if r > 0:
        cutouts.append(img_masked[xmin:xmax, ymin:ymax])

In [ ]:
for cutout in cutouts:
    plt.imshow(cutout)
    plt.show()

# Fitting the cutouts

In [ ]:
from astropy.modeling.models import Gaussian2D, Moffat2D
from photutils.aperture import CircularAperture, aperture_photometry


In [ ]:
popt_b =  [-2.70932858,   4.02468019,   0.61102148, -30.47160081,
         0.40609683]
gamma_moff, alpha_moff, A_moff, sigma_gauss, A_gauss = popt_b

psf_gauss_best = Gaussian2D(amplitude=A_gauss/(2*np.pi*sigma_gauss**2), x_stddev=sigma_gauss, y_stddev=sigma_gauss)
moff_norm = (alpha_moff - 1) / (np.pi * gamma_moff**2)
psf_moff_best = Moffat2D(amplitude=A_moff * moff_norm, alpha=alpha_moff, gamma=gamma_moff)

psf_model_2d = psf_gauss_best + psf_moff_best

In [ ]:
def fit_gauss_psf_chi2(image, uncertainty=None, mask=None, 
                 initial_scale=64_000, initial_shift=(0.0, 0.0),
                 bounds = None,
                 method='Nelder-Mead'):
    """
    Fit a PSF model to an image cutout by minimizing chi-squared.
    Optimizes only the shift (x, y) and amplitude scale of the PSF.
    
    Parameters
    ----------
    image : 2D array
        Image cutout to fit. Can contain NaN values.
    psf : 2D array
        PSF model to fit to the image.
    uncertainty : 2D array, optional
        Uncertainty/error map for the image. If None, assumes uniform weights.
    mask : 2D array of bool, optional
        Boolean mask where True indicates pixels to exclude from fit.
        If None, only NaN pixels are masked.
    initial_scale : float, optional
        Initial guess for PSF amplitude scale. Default is 1.0.
    initial_shift : tuple of float, optional
        Initial guess for PSF shift (dy, dx) in pixels. Default is (0, 0).
    method : str, optional
        Optimization method for scipy.optimize.minimize. Default is 'Nelder-Mead'.
    
    Returns
    -------
    result : dict
        Dictionary containing:
        - 'scale': Optimal amplitude scale factor
        - 'shift': Optimal shift (dy, dx) in pixels
        - 'chi2': Minimum chi-squared value
        - 'reduced_chi2': Reduced chi-squared (chi2 / dof)
        - 'model': Best-fit PSF model image
        - 'residual': Residual image (data - model)
        - 'success': Whether optimization converged
    """
    
    # Handle NaN pixels in image
    image_clean = np.copy(image)
    nan_mask = np.isfinite(image_clean)
    
    # Combine NaN mask with user-provided mask
    if mask is None:
        combined_mask = nan_mask
    else:
        combined_mask = nan_mask | mask
    
    # Set up uncertainty
    if uncertainty is None:
        uncertainty = 1.0

    
    # Calculate valid pixel count for degrees of freedom
    n_valid = np.sum(combined_mask)
    n_params = 5  # scale, dy, dx
    dof = max(1, n_valid - n_params)
    nX, nY = image.shape

    x_img, y_img = np.meshgrid(np.arange(nY), np.arange(nX))
    x_c, y_c = (nY - 1)/2, (nX-1)/2



    if bounds is None:
        bounds = (-nY/2, nY/2), (-nX/2, nX/2)
    def chi2_func(params):
        """Calculate chi-squared for given parameters."""
        scale, dx, dy, bg, sigma = params

        psf =  Gaussian2D(amplitude=scale/(2*np.pi*sigma**2), x_stddev=sigma, y_stddev=sigma)
        # Shift the PSF
        shifted_psf = psf(x_img - dx - x_c, y_img - dy - y_c)

        
        # Scale the PSF
        model = np.minimum(shifted_psf + bg, SAT_LEVEL)
        
        # Calculate residuals only for valid pixels
        residual = image_clean - model
        
        # Calculate chi-squared
        chi2 = np.sum(np.abs(residual[combined_mask])**2)
        
        return chi2
    
    # Initial parameters
    x0 = [initial_scale, initial_shift[0], initial_shift[1], 0, 0.65]
    
    # Optimize
    all_bounds = ((0, None), bounds[0], bounds[1], (-100, 100),  (0, 100))
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        opt_result = minimize(chi2_func, x0, method=method, bounds=all_bounds)
    
    # Extract best-fit parameters
    best_scale, best_dx, best_dy, best_bg, best_sigma = opt_result.x
    best_shift = (best_dx, best_dy)
    
    # Generate best-fit model
    psf =  Gaussian2D(amplitude=best_scale/(2*np.pi*best_sigma**2), x_stddev=best_sigma, y_stddev=best_sigma)
    shifted_psf = psf(x_img - best_dx - x_c, y_img - best_dy - y_c)
    best_model = np.minimum( shifted_psf + best_bg, SAT_LEVEL)
    
    # Calculate final residual
    residual = image_clean - best_model
    
    # Calculate final chi-squared
    min_chi2 = opt_result.fun
    reduced_chi2 = min_chi2 / dof
    
    return {
        'scale': best_scale,
        'shift': best_shift,
        'bkg': best_bg,
        'chi2': min_chi2,
        'reduced_chi2': reduced_chi2,
        'model': best_model,
        'residual': residual,
        'success': opt_result.success,
        'n_valid_pixels': n_valid,
        'dof': dof
    }


In [ ]:
def fit_model_psf_chi2(image, uncertainty=None, mask=None, 
                 initial_scale=1.0, initial_shift=(0.0, 0.0),
                 bounds = None,
                 method='Nelder-Mead'):
    """
    Fit a PSF model to an image cutout by minimizing chi-squared.
    Optimizes only the shift (x, y) and amplitude scale of the PSF.
    
    Parameters
    ----------
    image : 2D array
        Image cutout to fit. Can contain NaN values.
    psf : 2D array
        PSF model to fit to the image.
    uncertainty : 2D array, optional
        Uncertainty/error map for the image. If None, assumes uniform weights.
    mask : 2D array of bool, optional
        Boolean mask where True indicates pixels to exclude from fit.
        If None, only NaN pixels are masked.
    initial_scale : float, optional
        Initial guess for PSF amplitude scale. Default is 1.0.
    initial_shift : tuple of float, optional
        Initial guess for PSF shift (dy, dx) in pixels. Default is (0, 0).
    method : str, optional
        Optimization method for scipy.optimize.minimize. Default is 'Nelder-Mead'.
    
    Returns
    -------
    result : dict
        Dictionary containing:
        - 'scale': Optimal amplitude scale factor
        - 'shift': Optimal shift (dy, dx) in pixels
        - 'chi2': Minimum chi-squared value
        - 'reduced_chi2': Reduced chi-squared (chi2 / dof)
        - 'model': Best-fit PSF model image
        - 'residual': Residual image (data - model)
        - 'success': Whether optimization converged
    """
    
    # Handle NaN pixels in image
    image_clean = np.copy(image)
    nan_mask = np.isfinite(image_clean)
    
    # Combine NaN mask with user-provided mask
    if mask is None:
        combined_mask = nan_mask
    else:
        combined_mask = nan_mask | mask
    
    # Set up uncertainty
    if uncertainty is None:
        uncertainty = 1.0

    
    # Calculate valid pixel count for degrees of freedom
    n_valid = np.sum(combined_mask)
    n_params = 7  # scale, dy, dx
    dof = max(1, n_valid - n_params)
    nX, nY = image.shape

    x_img, y_img = np.meshgrid(np.arange(nY), np.arange(nX))
    x_c, y_c = (nY - 1)/2, (nX-1)/2



    if bounds is None:
        bounds = (-nY/2, nY/2), (-nX/2, nX/2)
    def chi2_func(params):
        """Calculate chi-squared for given parameters."""
        scale, dx, dy, alpha, gamma, A, sigma = params

        psf = Moffat2D(amplitude=scale, alpha=alpha, gamma=gamma) + Gaussian2D(amplitude=A/(2*np.pi*sigma**2), x_stddev=sigma, y_stddev=sigma)
        # Shift the PSF
        shifted_psf = psf(x_img - dx - x_c, y_img - dy - y_c)

        
        # Scale the PSF
        model = np.minimum(shifted_psf, SAT_LEVEL)
        
        # Calculate residuals only for valid pixels
        residual = image_clean - model
        
        # Calculate chi-squared
        chi2 = np.sum(np.abs(residual[combined_mask])**2)
        
        return chi2
    
    # Initial parameters
    x0 = [initial_scale, initial_shift[0], initial_shift[1], 2.5, 2.0, initial_scale, 0.65]
    
    # Optimize
    all_bounds = ((0, None), bounds[0], bounds[1], (0, None), (0, None), (0, None), (0, 100))
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        opt_result = minimize(chi2_func, x0, method=method, bounds=all_bounds)
    
    # Extract best-fit parameters
    best_scale, best_dx, best_dy, best_alpha, best_gamma, best_A, best_sigma = opt_result.x
    best_shift = (best_dx, best_dy)
    
    # Generate best-fit model
    psf = Moffat2D(amplitude=best_scale, alpha=best_alpha, gamma=best_gamma) + Gaussian2D(amplitude=best_A/(2*np.pi*best_sigma**2), x_stddev=best_sigma, y_stddev=best_sigma)
    shifted_psf = psf(x_img - best_dx - x_c, y_img - best_dy - y_c)
    best_model = np.minimum( shifted_psf, SAT_LEVEL)
    
    # Calculate final residual
    residual = image_clean - best_model
    
    # Calculate final chi-squared
    min_chi2 = opt_result.fun
    reduced_chi2 = min_chi2 / dof
    
    return {
        'scale': best_scale,
        'shift': best_shift,
        'alpha': best_alpha,
        'gamma': best_gamma,
        'chi2': min_chi2,
        'reduced_chi2': reduced_chi2,
        'model': best_model,
        'residual': residual,
        'success': opt_result.success,
        'n_valid_pixels': n_valid,
        'dof': dof
    }


In [ ]:
def fit_moff_psf_chi2(image, uncertainty=None, mask=None, 
                 initial_scale=1.0, initial_shift=(0.0, 0.0),
                 bounds = None,
                 method='Nelder-Mead'):
    """
    Fit a PSF model to an image cutout by minimizing chi-squared.
    Optimizes only the shift (x, y) and amplitude scale of the PSF.
    
    Parameters
    ----------
    image : 2D array
        Image cutout to fit. Can contain NaN values.
    psf : 2D array
        PSF model to fit to the image.
    uncertainty : 2D array, optional
        Uncertainty/error map for the image. If None, assumes uniform weights.
    mask : 2D array of bool, optional
        Boolean mask where True indicates pixels to exclude from fit.
        If None, only NaN pixels are masked.
    initial_scale : float, optional
        Initial guess for PSF amplitude scale. Default is 1.0.
    initial_shift : tuple of float, optional
        Initial guess for PSF shift (dy, dx) in pixels. Default is (0, 0).
    method : str, optional
        Optimization method for scipy.optimize.minimize. Default is 'Nelder-Mead'.
    
    Returns
    -------
    result : dict
        Dictionary containing:
        - 'scale': Optimal amplitude scale factor
        - 'shift': Optimal shift (dy, dx) in pixels
        - 'chi2': Minimum chi-squared value
        - 'reduced_chi2': Reduced chi-squared (chi2 / dof)
        - 'model': Best-fit PSF model image
        - 'residual': Residual image (data - model)
        - 'success': Whether optimization converged
    """
    
    # Handle NaN pixels in image
    image_clean = np.copy(image)
    nan_mask = np.isfinite(image_clean)
    
    # Combine NaN mask with user-provided mask
    if mask is None:
        combined_mask = nan_mask
    else:
        combined_mask = nan_mask | mask
    
    # Set up uncertainty
    if uncertainty is None:
        uncertainty = 1.0

    
    # Calculate valid pixel count for degrees of freedom
    n_valid = np.sum(combined_mask)
    n_params = 5  # scale, dy, dx
    dof = max(1, n_valid - n_params)
    nX, nY = image.shape

    x_img, y_img = np.meshgrid(np.arange(nY), np.arange(nX))
    x_c, y_c = (nY - 1)/2, (nX-1)/2



    if bounds is None:
        bounds = (-nY/2, nY/2), (-nX/2, nX/2)
    def chi2_func(params):
        """Calculate chi-squared for given parameters."""
        scale, dx, dy, alpha, gamma = params

        psf = Moffat2D(amplitude=scale, alpha=alpha, gamma=gamma) 
        # Shift the PSF
        shifted_psf = psf(x_img - dx - x_c, y_img - dy - y_c)

        
        # Scale the PSF
        model = np.minimum(shifted_psf, SAT_LEVEL)
        
        # Calculate residuals only for valid pixels
        residual = image_clean - model
        
        # Calculate chi-squared
        chi2 = np.sum(np.abs(residual[combined_mask])**2)
        
        return chi2
    
    # Initial parameters
    x0 = [initial_scale, initial_shift[0], initial_shift[1], 2.5, 2.0]
    
    # Optimize
    all_bounds = ((0, None), bounds[0], bounds[1], (0, None), (0, None))
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        opt_result = minimize(chi2_func, x0, method=method, bounds=all_bounds)
    
    # Extract best-fit parameters
    best_scale, best_dx, best_dy, best_alpha, best_gamma = opt_result.x
    best_shift = (best_dx, best_dy)
    
    # Generate best-fit model
    psf = Moffat2D(amplitude=best_scale, alpha=best_alpha, gamma=best_gamma)
    shifted_psf = psf(x_img - best_dx - x_c, y_img - best_dy - y_c)
    best_model = np.minimum( shifted_psf, SAT_LEVEL)
    
    # Calculate final residual
    residual = image_clean - best_model
    
    # Calculate final chi-squared
    min_chi2 = opt_result.fun
    reduced_chi2 = min_chi2 / dof
    
    return {
        'scale': best_scale,
        'shift': best_shift,
        'alpha': best_alpha,
        'gamma': best_gamma,
        'chi2': min_chi2,
        'reduced_chi2': reduced_chi2,
        'model': best_model,
        'residual': residual,
        'success': opt_result.success,
        'n_valid_pixels': n_valid,
        'dof': dof
    }


In [ ]:
global_bkg = np.median(img_err)

In [ ]:
for img in cutouts:
    psf_fit = fit_moff_psf_chi2(img, psf_model_2d, initial_scale=100_000)
    print(psf_fit["chi2"], psf_fit["success"], psf_fit["scale"], psf_fit["alpha"], psf_fit["gamma"])

    
    fig, axs = plt.subplots(1, 3, layout="constrained", figsize=(6, 4))
    plt.sca(axs[0])
    vmin, vmax = -10, 10
    
    img_kwargs = dict(origin="lower", vmin=vmin, vmax=vmax, cmap="RdBu", norm="asinh")
    plt.imshow(img/global_bkg, **img_kwargs)
    plt.title("image")

    shift = psf_fit["shift"]
    x_c, y_c = (img.shape[1]-1)/2, (img.shape[0]-1)/2
    plt.scatter(x_c + shift[0], y_c + shift[1])



    plt.sca(axs[1])
    plt.imshow(psf_fit["model"]/global_bkg, **img_kwargs)
    plt.title("ana psf model")

    plt.sca(axs[2])
    p = plt.imshow(psf_fit["residual"]/global_bkg, **img_kwargs)
    plt.title("ana psf residual")
    plt.show()

In [ ]:
Moffat2D?